In [ ]:
%pylab notebook
import matplotlib.pyplot as plt
from charistools.meltModels import TriSurfTempIndexMelt
from charistools.meltModels import ImshowTriSurfMelt
from charistools.meltModels import PlotTriSurfInput
from charistools.meltModels import PlotTriSurfMelt
from charistools.modelEnv import ModelEnv
import re
import os

In [ ]:
configFile = '/Users/brodzik/projects/CHARIS/charistools_test_data/calibration_modelEnv_config.ini'
myEnv = ModelEnv(tileConfigFile=configFile)


In [ ]:
#drainageID = 'IN_Hunza_at_DainyorBridge'
drainageID = 'AM_Vakhsh_at_Komsomolabad'
%cd ~/projects/CHARIS/charistools_test_data/derived_hypsometries/$drainageID
%pwd

In [ ]:
ablation_method = 'albedo_mod10a1'
threshold = 0.40
input = myEnv.model_inputs(drainageID=drainageID,
                           year=2001,
                           ablation_method=ablation_method,
                           threshold=threshold)

In [ ]:
#input

In [ ]:
best_model = "8.0_9.44_8.0_15.0"
best_model = "8.89_9.0_12.0_19.0"
best_model = "4.0_5.0_4.0_11.0"
(min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf) = best_model.split("_")
min_snow_ddf = float(min_snow_ddf)
max_snow_ddf = float(max_snow_ddf)
min_ice_ddf = float(min_ice_ddf)
max_ice_ddf = float(max_ice_ddf)
print(min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf)

In [ ]:
(SOLmelt, SOImelt, EGImelt) = TriSurfTempIndexMelt(
    input['snow_on_land_by_elevation_filename'],
    input['snow_on_ice_by_elevation_filename'],
    input['exposed_glacier_ice_by_elevation_filename'],
    input['temperature_by_elevation_filename'],
    min_snow_ddf=min_snow_ddf,
    max_snow_ddf=max_snow_ddf,
    min_ice_ddf=min_ice_ddf,
    max_ice_ddf=max_ice_ddf)

In [ ]:
import os
import re
p = re.compile(drainageID)
outdir = '/Users/brodzik/projects/CHARIS/charistools_test_data/derived_hypsometries/' + drainageID
SOLmeltfile = os.path.basename(input['snow_on_land_by_elevation_filename'])
SOLmeltfile = SOLmeltfile.replace('_area_by_elev', '_melt_by_elev')
SOLmeltfile = outdir + '/' + p.sub(drainageID + '.model_' + best_model, SOLmeltfile)

SOImeltfile = os.path.basename(input['snow_on_ice_by_elevation_filename'])
SOImeltfile = SOImeltfile.replace('_area_by_elev', '_melt_by_elev')
SOImeltfile = outdir + '/' + p.sub(drainageID + '.model_' + best_model, SOImeltfile)

EGImeltfile = os.path.basename(input['exposed_glacier_ice_by_elevation_filename'])
EGImeltfile = EGImeltfile.replace('_area_by_elev', '_melt_by_elev')
EGImeltfile = outdir + '/' + p.sub(drainageID + '.model_' + best_model, EGImeltfile)

print(SOLmeltfile)
print(SOImeltfile)
print(EGImeltfile)

In [ ]:
from charistools.hypsometry import Hypsometry
SOLmelt.write(filename=SOLmeltfile, decimal_places=2, verbose=True)
SOImelt.write(filename=SOImeltfile, decimal_places=2, verbose=True)
EGImelt.write(filename=EGImeltfile, decimal_places=2, verbose=True)

In [ ]:
outfile = SOImeltfile
outfile
p = re.compile(r'snow_on_ice.+')
outfile = p.sub('melt_tseries.png', outfile)
outfile

In [ ]:
fig, axes = plt.subplots(3,1)
ImshowTriSurfMelt(axes, SOLmelt, SOImelt, EGImelt)
fig.tight_layout()

In [ ]:
rainfallFile = myEnv.calibration_filename(type="rainfall", 
                                          drainageID=drainageID)
runoffFile = myEnv.calibration_filename(type="runoff", 
                                          drainageID=drainageID)


In [ ]:
from charistools.timeSeries import TimeSeries
rainfall = TimeSeries(rainfallFile)
runoff = TimeSeries(runoffFile)

In [ ]:
total_rainfall = rainfall.data['rainfall']['2001-01-01':'2001-12-01'].sum()
total_runoff = runoff.data['runoff']['2001-01-01':'2001-12-01'].sum()
print("total rainfall = %.2f" % total_rainfall)
print("total runoff = %.2f" % total_runoff)

In [ ]:
melt_by_doy = (                                                             
        SOLmelt.data_by_doy() +                                           
        SOImelt.data_by_doy() +                                           
        EGImelt.data_by_doy())
total_melt = melt_by_doy.sum()
print("total melt = %.2f" % total_melt)

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(9,8))
ax[0] = PlotTriSurfInput(ax[0], 
                         input['snow_on_land_by_elevation_hyps'],
                         input['snow_on_ice_by_elevation_hyps'],
                         input['exposed_glacier_ice_by_elevation_hyps'],
                         input['temperature_by_elevation_hyps'],
                         title="Inputs for " + drainageID)
ax[1] = PlotTriSurfMelt(ax[1], SOLmelt, SOImelt, EGImelt, 
                     title=drainageID + " 2001")
ax[1].text('2001-01-15', ax[1].get_ylim()[1] * 0.8,
           'melt + rainfall = %.2f $km^2$\nrunoff = %.2f $km^2$' % 
           (total_melt + total_rainfall, total_runoff), 
           style='italic',
            bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
ax[1].text('2001-01-15', ax[1].get_ylim()[0] + 
           (0.1 * (ax[1].get_ylim()[1] - ax[1].get_ylim()[0])),
           'snow DDF = %.2f - %.2f $mm/C/day$\nice DDF = %.2f - %.2f $mm/C/day$' % 
           (min_snow_ddf, max_snow_ddf, min_ice_ddf, max_ice_ddf),
           style='italic',
            bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
fig.tight_layout()
fig.savefig(outfile)
